# OrthodoxAI - RAG

> Konstantinos Mpouros <br>
> Github: https://github.com/konstantinosmpouros?tab=repositories<br>
> Year: 2025

## About the Project

This notebook is dedicated to prototyping and testing **Retrieval-Augmented Generation (RAG)** methods within the **OrthodoxAI** framework.

It serves as a sandbox to:

- Evaluate different libraries and methods for building vector stores  
- Benchmark the performance and timing of the retrieval process  
- Isolate and debug components before integrating them into the main agent system

This notebook is focused solely on the retrieval layer of RAG, without involving agent roles or generation logic at this stage.

## Libraries

In [1]:
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

## Chroma Vector Store

### OpenAI

* Create the vector store

In [25]:
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import JSONLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings

from langchain.vectorstores import Chroma
import json

In [3]:
JSON_DIR = "./knowledge_base/Orthodox/Omilies/speeches_athanasios_mitilinaios/"

loader = DirectoryLoader(
    JSON_DIR,
    glob="*.json",
    loader_cls=JSONLoader,
    loader_kwargs={"jq_schema": ".content"},
)
documents = loader.load()

In [4]:
documents[-1]

Document(metadata={'source': '/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/3165_ΑΠΑΝΤΗΣΕΙΣ_ΑΠΟΡΙΩΝ_ΑΝΩΤ_ΚΑΤΗΧΗΤΙΚΟΥ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json', 'seq_num': 1}, page_content="Και πηγαίνουμε πιο κάτω. Ποια είναι η στάση των ανθρώπων μπροστά στο θάνατο; Τι θα μπορούσαμε να πούμε πάνω σε αυτό; Ξέρετε ότι σήμερα οι άνθρωποι στέκονται μπροστά στο θάνατο, να πω τη φράση του Αποστόλου Παύλου; Ως άθεοι εν κόσμω ελπίδα μη έχοντες. Ξέρετε τι σημαίνει αυτό; Αγνοούν την ανάσταση των νεκρών. Ως άθεοι εν κόσμω, ελπίδα μη έχοντες. Τι ελπίδα; Αναστάσεως των νεκρών. Τι κρίμα! Λέει ο Απόστολος Παύλος στους Θεσσαλονικείς , Δεν θέλω να λυπόσαστε όπως και οι λοιποί οι μη έχοντες ελπίδα, αυτοί που δεν έχουν ελπίδα αναστάσεως των νεκρών. Παιδιά, η στάση των ανθρώπων μπροστά στο θάνατο, των ποιών ανθρώπων; Των σωστών, των Χριστιανών, είναι ότι ο Χριστός ανεστήθη και εφόσον ανεστήθη ο Χριστός, θα αναστηθούν και οι νεκροί. Και ο Χριστός είναι ο πρωτότ

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

In [6]:
chunks[0].page_content

"Πρέπει να ευχαριστήσουμε τον Άγιο Τριαδικό Θεό μας, παιδιά , που μας αξίωσε πάλι να επανέλθουμε εις τον τόπον αυτόν, να ακουστεί ο Θείος του Λόγος. . Στην Αρχαία Εκκλησία η Βάπτισις εγίνεται εις μεγάλην ηλικίαν. Επειδή τώρα υπάρχει ο νηπιο-νηπιοβαπτισμός, η κατήχηση, όπως καταλαβαίνετε, γίνεται-γίνεται η κατήχηση, γίνεται μετά το βάπτισμά μας. Γι' αυτό το λόγο είναι μια αληθινή ευλογία αν κάποτε καταλάβουμε ότι πρέπει να συμπληρώσουμε την κατήχησή μας μετά το βάπτισμά μας, όταν μεγαλώσουμε. Δεν είναι ούτε πολυτέλεια η κατήχηση, ούτε κάτι το έκτακτο ή το, αν θέλετε, το προαιρετικό. Η κατήχηση είναι αναγκαιοτάτη. Γι' αυτό το λόγο, άμα συνειδητο-συνειδητοποιήσομε ότι πρέπει να αποκτήσωμε τη γνώση του περιεχομένου της πίστεώς μας, τότε αναμφισβήτητα θα τρέχουμε όχι στα παιδικά μας και εφηβικά μας χρόνια να συμπληρώνουμε τη γνώση μας, την πνευματική, τη θεολογική, τη χριστιανική, αλλά σε ολόκληρη τη ζωή μας. Οι μαθηταί του Χριστού, μέχρι που πέθαναν, ελέγοντο μαθηταί, που σημαίνει είχανε"

In [7]:
len(chunks)

8574

In [12]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large", chunk_size=10) # nomic-embed-text-v1 is god 

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="vectorstore/chroma_db_openai",
    collection_name="athanasios-muthlinaios",
)
vectordb.persist()

/tmp/ipykernel_9623/4027366378.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [8]:
del vectordb

* Reload the vector store and query the data

In [3]:
# Re-load your persisted DB
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    embedding_function=embedding_model,
    persist_directory="vectorstore/chroma_db_openai",
    collection_name="athanasios-muthlinaios",
)

/tmp/ipykernel_7236/4170041564.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [4]:
query = "What did Athanasios Mytilenaios say about humidity?"
results = vectordb.similarity_search_with_score(query, k=5)

# Inspect docs and their distances
for i, (doc, distance) in enumerate(results):
    print(f"--- Result {i} ---")
    print(f"Similarity (higher is better): {distance:.4f}")
    print("Content:", doc.page_content.replace("\n", " "), "…")
    print("Metadata:", doc.metadata)
    print()

--- Result 0 ---
Similarity (higher is better): 1.3464
Content: κάποιους επισκέπτες και δεν είχε νερό. Χρειάστηκε μια κατσαρόλα νερό να βράσει λίγα όσπρια για να τους φυλάξει. Δεν είχε νερό. Τότε κάνει μια προσευχή και λέει. Κύριε του ουρανού και της γης, Συ που δίνεις των ηετών τη βροχή από τον ουρανό, δώσε μου λίγο νερό να βράσω τα όσπρια, για να φυλάξω τους επισκέπτες μου. Πριν τελειώσει την προσευχή του αμέσως ένα συννεφάκι από πάνω το συννεφάκι αυτό άρχισε να βρέχει μόνο στο σημείο που ήταν ο ασκητής και απέκτησε η κατσαρόλα του τόσο νερό όσο χρειάστηκε για να βράσει τα όσπρια. Κι όμως δεν έβρεξε σε όλη την περιοχή, παρά μόνο σε εκείνο το σημείο που χρειάστηκε να μαζέψει ο ασκητής λίγο νεράκι. Για το Θεό αγαπητοί μου, για το Θεό υπάρχει αδυναμία; Βγήκε λοιπόν άνεμος από τη θάλασσα δηλαδή από τον Ελαμιτικών κόλπον. Ήρθε ο άνεμος αυτός και παρέσυρε ορτιγομήτραν. Πουλιά. Είναι γνωστό ότι τα ορτύκια, ορτιγομήτρα λέγεται δηλαδή ορτυγομάνα θα το λέγαμε σήμερα στη γλώσσα μας, δηλαδή πλήθ

In [5]:
retriever = vectordb.as_retriever(search_kwargs={"k": 12})

In [6]:
query = "What did Athanasios Mytilenaios say about humidity?"
results = retriever.invoke(query)

# Inspect docs and their distances
for i, doc in enumerate(results):
    print(f"--- Result {i} ---")
    print("Content:", doc.page_content.replace("\n", " "))
    print("Metadata:", doc.metadata)
    print()

--- Result 0 ---
Content: κάποιους επισκέπτες και δεν είχε νερό. Χρειάστηκε μια κατσαρόλα νερό να βράσει λίγα όσπρια για να τους φυλάξει. Δεν είχε νερό. Τότε κάνει μια προσευχή και λέει. Κύριε του ουρανού και της γης, Συ που δίνεις των ηετών τη βροχή από τον ουρανό, δώσε μου λίγο νερό να βράσω τα όσπρια, για να φυλάξω τους επισκέπτες μου. Πριν τελειώσει την προσευχή του αμέσως ένα συννεφάκι από πάνω το συννεφάκι αυτό άρχισε να βρέχει μόνο στο σημείο που ήταν ο ασκητής και απέκτησε η κατσαρόλα του τόσο νερό όσο χρειάστηκε για να βράσει τα όσπρια. Κι όμως δεν έβρεξε σε όλη την περιοχή, παρά μόνο σε εκείνο το σημείο που χρειάστηκε να μαζέψει ο ασκητής λίγο νεράκι. Για το Θεό αγαπητοί μου, για το Θεό υπάρχει αδυναμία; Βγήκε λοιπόν άνεμος από τη θάλασσα δηλαδή από τον Ελαμιτικών κόλπον. Ήρθε ο άνεμος αυτός και παρέσυρε ορτιγομήτραν. Πουλιά. Είναι γνωστό ότι τα ορτύκια, ορτιγομήτρα λέγεται δηλαδή ορτυγομάνα θα το λέγαμε σήμερα στη γλώσσα μας, δηλαδή πλήθος, πλήθος, πάρα πολύ πλήθος. Όχι απλώ

In [21]:
query = "What did Athanasios Mytilenaios say about humidity?"
results = retriever.invoke(query)
retrieved_docs = []

# Inspect docs and their distances
for i, doc in enumerate(results):
    retrieved_docs.append({"Content:": doc.page_content.replace("\n", " "),
                            "Metadata:": doc.metadata})

In [27]:
retrieved_docs

[{'Content:': 'Ποτέ. Τι λέγει ο Ψαλμωδός; Κύριος Ποίμεν με και ουδέν με υστερήσει. Εις τόπον χλωρίς ευχή με κατεσκήνωσε. Κύριος είναι ο τσομπάνης μου, εγώ είμαι το κορβάδι του. Ο Κύριος με τιμαίνει και σε τίποτε δεν θα με υστερήσει. Τι έχω να φοβηθώ; Τι έχω να πάθω; Τίποτα. Γιατί; Γιατί ο Θεός είναι ο Πατέρας μου και ο Πατέρας ολοκλήρου της Δημιουργίας.',
  'Metadata:': {'chunk_id': '/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json_chunk_426',
   'seq_num': 1,
   'source': '/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json'}},
 {'Content:': 'είναι ο αντίχριστος. Πότε; Όταν οι άνθρωποι δεν θα αναγνωρίζουν τον Αντίχριστο και θα πλανώνται, θα το θεωρούν ότι είναι μέγας φιλάνθρωπος. Θέλετε μια αποδείξει; Αγαπητοί μου, αυτή την εποχή τώρα ζούμε τους προδρόμους του αντιχρίστου

In [29]:
str(retrieved_docs)

"[{'Content:': 'Ποτέ. Τι λέγει ο Ψαλμωδός; Κύριος Ποίμεν με και ουδέν με υστερήσει. Εις τόπον χλωρίς ευχή με κατεσκήνωσε. Κύριος είναι ο τσομπάνης μου, εγώ είμαι το κορβάδι του. Ο Κύριος με τιμαίνει και σε τίποτε δεν θα με υστερήσει. Τι έχω να φοβηθώ; Τι έχω να πάθω; Τίποτα. Γιατί; Γιατί ο Θεός είναι ο Πατέρας μου και ο Πατέρας ολοκλήρου της Δημιουργίας.', 'Metadata:': {'chunk_id': '/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json_chunk_426', 'seq_num': 1, 'source': '/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json'}}, {'Content:': 'είναι ο αντίχριστος. Πότε; Όταν οι άνθρωποι δεν θα αναγνωρίζουν τον Αντίχριστο και θα πλανώνται, θα το θεωρούν ότι είναι μέγας φιλάνθρωπος. Θέλετε μια αποδείξει; Αγαπητοί μου, αυτή την εποχή τώρα ζούμε τους προδρόμους του αντιχρίστου και βλέ

In [32]:
json.dumps(retrieved_docs, ensure_ascii=False, indent=2)

'[\n  {\n    "Content:": "Ποτέ. Τι λέγει ο Ψαλμωδός; Κύριος Ποίμεν με και ουδέν με υστερήσει. Εις τόπον χλωρίς ευχή με κατεσκήνωσε. Κύριος είναι ο τσομπάνης μου, εγώ είμαι το κορβάδι του. Ο Κύριος με τιμαίνει και σε τίποτε δεν θα με υστερήσει. Τι έχω να φοβηθώ; Τι έχω να πάθω; Τίποτα. Γιατί; Γιατί ο Θεός είναι ο Πατέρας μου και ο Πατέρας ολοκλήρου της Δημιουργίας.",\n    "Metadata:": {\n      "chunk_id": "/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json_chunk_426",\n      "seq_num": 1,\n      "source": "/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json"\n    }\n  },\n  {\n    "Content:": "είναι ο αντίχριστος. Πότε; Όταν οι άνθρωποι δεν θα αναγνωρίζουν τον Αντίχριστο και θα πλανώνται, θα το θεωρούν ότι είναι μέγας φιλάνθρωπος. Θέλετε μια αποδείξει; Αγαπητοί μου, αυτή την 

In [33]:
type(json.dumps(retrieved_docs, ensure_ascii=False, indent=2))

str

### Ollama Nomic

* Create the vector store

In [2]:
from langchain_community.document_loaders import JSONLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from tqdm.auto import tqdm

In [3]:
# Load & split your transcripts
dir_path = "./knowledge_base/Orthodox/Omilies/speeches_athanasios_mitilinaios/"
loader = DirectoryLoader(
    dir_path,
    glob="*.json",
    loader_cls=JSONLoader,
    loader_kwargs={"jq_schema": ".content"},
)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)
print(f"Total chunks to embed: {len(chunks)}")

Total chunks to embed: 8574


In [4]:
# Initialize Chroma DB with Ollama embeddings
persist_dir = "vectorstore/chroma_db_ollama_nomic"
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://127.0.0.1:11434",
)
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory=persist_dir,
    collection_name="athanasios-muthlinaios",
)

# Embed & add each chunk with tqdm progress bar
for i, chunk in enumerate(tqdm(chunks, desc="Embedding chunks"), start=1):
    chunk_id = f"{chunk.metadata.get('source', 'doc')}_chunk_{i}"
    chunk.metadata["chunk_id"] = chunk_id
    vectordb.add_documents([chunk], ids=[chunk_id])

print(f"✅ All {len(chunks)} chunks embedded via nomic-embed-text-v1 and saved.")

Embedding chunks:   0%|          | 0/8574 [00:00<?, ?it/s]

✅ All 8574 chunks embedded via nomic-embed-text-v1 and saved.


In [14]:
del vectordb

* Reload the vectore store and query the data

In [15]:
embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
    base_url="http://127.0.0.1:11434",
)

vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory="vectorstore/chroma_db_ollama_nomic",
    collection_name="athanasios-muthlinaios",
)

In [16]:
docs = vectordb.similarity_search(
    "What did Athanasios Mytilenaios say about humidity?",
    k=5
)
for i, doc in enumerate(docs, 1):
    print(f"--- Result {i} ---")
    print(doc.page_content.replace("\n", " "))
    print("► Source:", doc.metadata.get("source"))
    print("► Chunk ID:", doc.metadata.get("chunk_id"))
    print()

--- Result 1 ---
Ποτέ. Τι λέγει ο Ψαλμωδός; Κύριος Ποίμεν με και ουδέν με υστερήσει. Εις τόπον χλωρίς ευχή με κατεσκήνωσε. Κύριος είναι ο τσομπάνης μου, εγώ είμαι το κορβάδι του. Ο Κύριος με τιμαίνει και σε τίποτε δεν θα με υστερήσει. Τι έχω να φοβηθώ; Τι έχω να πάθω; Τίποτα. Γιατί; Γιατί ο Θεός είναι ο Πατέρας μου και ο Πατέρας ολοκλήρου της Δημιουργίας.
► Source: /mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json
► Chunk ID: /mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json_chunk_426

--- Result 2 ---
είναι ο αντίχριστος. Πότε; Όταν οι άνθρωποι δεν θα αναγνωρίζουν τον Αντίχριστο και θα πλανώνται, θα το θεωρούν ότι είναι μέγας φιλάνθρωπος. Θέλετε μια αποδείξει; Αγαπητοί μου, αυτή την εποχή τώρα ζούμε τους προδρόμους του αντιχρίστου και βλέπετε τους Χριστιανούς μας και δια

In [17]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
results = retriever.get_relevant_documents("Tell me about moisture in his speeches")
for i, doc in enumerate(results, 1):
    print(f"[{i}]", doc.page_content.replace("\n", " "), "…")
    print("   →", doc.metadata.get("source"), doc.metadata.get("chunk_id"))
    print()

[1] αυτή η περιέργεια. Είναι εκείνο που έλεγαν οι αρχαίοι Έλληνες, ότι «φύσει άνθρωπος ορέγετε του είδεναι». Εκ φύσεως ο άνθρωπος επιθυμεί να γνωρίζει. Δηλαδή αυτή η επιθυμία να γνωρίζει, η περιέργεια να γνωρίσει. Η περιέργεια ως τάχης γνωστική θα έκανε βέβαια τους πρωτοπλάστους να γνωρίσουν βαθύτερα και βαθύτερα τον δημιουργό τους, αλλά το είδος είναι φιλαργύρευμα.Και με τα πτωτικά ακόμα δεν θα ήταν παρά μία η περιέργεια, ή καλώς νοούμενοι, παρά μια αναζήτησις του αρχετύπου των, δηλαδή του ενανθρωπήσαντος Θεού Λόγου. Έτσι η περιέργεια με την καλή της, την αγαθή της διάσταση εμφανίζεται ως φιλομάθεια, σαν μια δίψα της ψυχής, μια δίψα του ανθρώπου για τη γνώση, για την επιστήμη. Λέγει θαυμάσια ο Ισοκράτης, στον Δημώνικο, του γράφει «Εάν εισ φιλομαθής, έστι και πολυμαθής». Εάν, λέγει, εε, είσαι φιλομαθής, αγαπάς να μαθαίνεις, τότε θα γίνεις και πολυμαθής. Θα γνωρίσεις πολλά πράγματα ακόμη «και αν μεν επίστασαι φίλασαι ταις μελέταις, αν δε μη ναι μάθηκας προσλάμβανεται τις επιστήμινες», …

### Ollama MXBAI

* Create the vector store

In [2]:
from langchain_community.document_loaders import JSONLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from tqdm.auto import tqdm

In [3]:
# Load & split your transcripts
dir_path = "./knowledge_base/Orthodox/Omilies/speeches_athanasios_mitilinaios/"
loader = DirectoryLoader(
    dir_path,
    glob="*.json",
    loader_cls=JSONLoader,
    loader_kwargs={"jq_schema": ".content"},
)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)
print(f"Total chunks to embed: {len(chunks)}")

Total chunks to embed: 8574


In [5]:
# Initialize Chroma DB with Ollama embeddings
persist_dir = "vectorstore/chroma_db_ollama_mxbai"
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
    base_url="http://127.0.0.1:11434",
)
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory=persist_dir,
    collection_name="athanasios-muthlinaios",
)

# Embed & add each chunk with tqdm progress bar
for i, chunk in enumerate(tqdm(chunks, desc="Embedding chunks"), start=1):
    chunk_id = f"{chunk.metadata.get('source', 'doc')}_chunk_{i}"
    chunk.metadata["chunk_id"] = chunk_id
    vectordb.add_documents([chunk], ids=[chunk_id])

print(f"✅ All {len(chunks)} chunks embedded via nomic-embed-text-v1 and saved.")

Embedding chunks:   0%|          | 0/8574 [00:00<?, ?it/s]

✅ All 8574 chunks embedded via nomic-embed-text-v1 and saved.


In [6]:
del vectordb

* Reload the vectore store and query the data

In [10]:
embeddings = OllamaEmbeddings(
    model="mxbai-embed-large",
    base_url="http://127.0.0.1:11434",
)

vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory="vectorstore/chroma_db_ollama_mxbai",
    collection_name="athanasios-muthlinaios",
)

In [11]:
docs = vectordb.similarity_search(
    "What did Athanasios Mytilenaios say about humidity?",
    k=5
)
for i, doc in enumerate(docs, 1):
    print(f"--- Result {i} ---")
    print(doc.page_content.replace("\n", " "))
    print("► Source:", doc.metadata.get("source"))
    print("► Chunk ID:", doc.metadata.get("chunk_id"))
    print()

--- Result 1 ---
ο γιος της οικογενείας που έρχεται η κόρη τώρα μέσα στο σπίτι και ομοίως να γίνει-- να γίνεται κατάθεσις των χρημάτων εις τους γονείς και θα πρέπει τώρα η νύφη, ακόμα χειρότερο αυτό, να πάει να πει στον πεθερό πατέρα, ξέρεις, θέλω μια φούστα, θέλω ένα φουστάνι.Και ο πατέρας να είναι εκείνος που θα κρίνει αν θα πρέπει να αγοράσει η νύφη καινούργιο φουστάνι ή όχι και όχι ο σύζυγος. Αυτό συμβαίνει σε πολλά σπίτια. Αγαπητοί μου. Ερωτώ πάλι Μπορεί να αισθάνεται μια τέτοια κοπέλα ένας τέτοιος νέος μέσα στο σπίτι, σε ένα τέτοιο σπίτι άνετα και να ωριμάσουν σαν πρόσωπα; Ποτέ. Αν υποτεθεί ότι ή εσείς θα καλέσετε ένα παιδί να γίνει στο σπίτι σας γαμπρός ή δεν ξέρω πώς, πάντως εγώ να πω τούτο. Να μάθουμε αγαπητοί μου το εξής Μπορούμε να βοηθήσουμε τα παιδιά μας, αλλά γρήγορα θα αποσυρθούν και θα κυβερνήσουν τα παιδιά μας μέσα στο σπίτι. Εάν φοβείστε ότι δεν θα δύναστε να κρατάτε εσείς εκείνα που θέλετε να κρατάτε, τουλάχιστον μερικά οικονομικά, θα σας έλεγα Κρατήστε ένα μέρος της

In [13]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})
results = retriever.get_relevant_documents("Tell me about moisture in his speeches")
for i, doc in enumerate(results, 1):
    print(f"[{i}]", doc.page_content.replace("\n", " "), "…")
    print("   →", doc.metadata.get("source"), doc.metadata.get("chunk_id"))
    print()

[1] ο Κύριος, ο Θεός του Παραδείσου. Για σκεφτείτε αυτό το πράγμα. Σκεφτείτε τι μεγάλη τιμή έχει ο άνθρωπος. Αυτά που τα λέμε τόσο απλά. Ξέρετε τι μεγάλη αξία έχουν; Ξέρετε ότι σήμερα οι άνθρωποι προσπαθούν να δώσουν, να κατανοήσουν με τη φιλοσοφία τι είναι ο άνθρωπος; Και όταν κάποτε ανακαλύπτουν την αξία άνθρωπος διακηρύσσουν, ξέρετε, λέει, ο άνθρωπος είναι πρόσωπον. Χαίρω πολύ, θα τους λέγαμε. Είναι πρόσωπον. Τώρα το ανακαλύψατε. Ναι, λέει, δεν είναι μάζα. Είναι πρόσωπον ο άνθρωπος. Μα αυτό το ξέρουμε εμείς από την αποκάλυψη του Θεού. Όχι μόνο ότι ο άνθρωπος δεν είναι πρόσωπον, αλλά είναι ένας μικρός θεός επάνω στη γη, με προορισμό να θεωρηθεί. Και όταν αργότερα ο διάβολος θα του πει του Αδάμ Σας είπε ο Θεός να μη δοκιμάσετε από τον καρπόν για να μη γίνετε θεοί, εδώ έλεγε ένα ψέμα ο διάβολος αλλά και μιαν αλήθεια. Η αλήθεια είναι ότι πραγματικά θα εγίνω το θεοί. Όχι όμως τρώγοντας τον καρπό του δέντρου που ο Θεός είπε να μη δοκιμάσω, αλλά θα εγίνουν το θεοί έχοντας την εξάρτησή τους

### Hybrid Search

In [ ]:
from langchain.retrievers import HybridSearchRetriever
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever  # e.g. from langchain.retrievers import BM25Retriever

# 1) your vector-store retriever
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    embedding_function=embeddings,
    persist_directory="vectorstore/chroma_db_openai",
    collection_name="athanasios-muthilinaios",
)
vector_retriever = vectordb.as_retriever(search_kwargs={"k": 5})

# 2) a simple BM25 / keyword retriever over the same docs 
#    (you can point it at the same text source you used to build Chroma)
loader   = DirectoryLoader(JSON_DIR, glob="*.json", loader_cls=JSONLoader, loader_kwargs={"jq_schema": ".content"})
docs     = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks   = splitter.split_documents(docs)
bm25     = BM25Retriever.from_documents(chunks)    # which gives you a .get_relevant_documents()

# 3) glue them together
hybrid = HybridSearchRetriever.from_retrievers([
    vector_retriever, 
    bm25
], weights=[0.7, 0.3])  # e.g. 70% vector score + 30% BM25 score

# 4) run your query
query   = "What did Athanasios Mytilenaios say about humidity?"
results = hybrid.get_relevant_documents(query)

for i, doc in enumerate(results):
    print(f"--- Result {i} ---")
    print(doc.page_content)
    print(doc.metadata)
    print()


## Chroma Client

In [66]:
import os
import asyncio

from fastapi import HTTPException

import chromadb
from chromadb.config import Settings
from langchain_chroma import Chroma

from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document

In [67]:
PORT = 8000
COLLECTION = 'athanasios-muthlinaios'

In [68]:
embeddings_model = OpenAIEmbeddings(model='text-embedding-3-large')

settings = Settings(
    chroma_api_impl="rest",
    chroma_server_host='localhost',
    chroma_server_http_port=PORT
)

In [69]:
async def retrieve(request: str, k: int):
    client = chromadb.HttpClient(
        host='localhost',
        port=int(PORT),
        settings=settings
    )
    vectordb = Chroma(
        client=client,
        collection_name=COLLECTION,
        embedding_function=embeddings_model,
    )
    
    retriever = vectordb.as_retriever(search_kwargs={"k": k})
    docs: list[Document] = await retriever.ainvoke(request)
    return {
        "query": request,
        "k": k,
        "documents": [
            {"text": d.page_content, "metadata": d.metadata} for d in docs
        ],
    }

In [70]:
result = await retrieve("What is Psalm 23?", 5)
print(result)

{'query': 'What is Psalm 23?', 'k': 5, 'documents': [{'text': 'Ποτέ. Τι λέγει ο Ψαλμωδός; Κύριος Ποίμεν με και ουδέν με υστερήσει. Εις τόπον χλωρίς ευχή με κατεσκήνωσε. Κύριος είναι ο τσομπάνης μου, εγώ είμαι το κορβάδι του. Ο Κύριος με τιμαίνει και σε τίποτε δεν θα με υστερήσει. Τι έχω να φοβηθώ; Τι έχω να πάθω; Τίποτα. Γιατί; Γιατί ο Θεός είναι ο Πατέρας μου και ο Πατέρας ολοκλήρου της Δημιουργίας.', 'metadata': {'seq_num': 1, 'source': '/mnt/c/Users/user/Desktop/OrthodoxAI/src/multi_agent/rag/knowledge_base/Omilies/speeches_athanasios_mitilinaios/0012_23-01-83_ΚΟΣΜΟΛΟΓΙΑ_π_ΑΘ_ΜΥΤΙΛΗΝΑΙΟΥ.json'}}, {'text': 'Να πάω στον ουρανό; Είσαι εκεί. Να κατέβω στα έγκατα της θαλάσσης; Είσαι εκεί. Να πάω στον Άδη; Είσαι παρών. Πού να πάω; Όπου και να πάω είσαι παρών. Αυτή η αίσθησις της πανταχού παρουσίας του Θεού. Αυτή η αίσθησις της προνοίας του Θεού. Αυτή η αίσθησις της Κυβερνήσεως του Θεού. Αυτή η αίσθησις της αγάπης του Θεού κάνει τον άνθρωπο να αισθάνεται καθόλου μοναξιά, να βρίσκει νόημα σ

## HR-Policies Vectore Stores

### HR-Policies VDB vol1

* Create the vector store

In [2]:
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings

from langchain.vectorstores import Chroma
import json

In [3]:
PDF_DIR = "./knowledge_base/HR-Policies V1/"

loader = DirectoryLoader(
    PDF_DIR,
    glob="**/*.pdf",
    loader_cls=PyPDFLoader,
)
documents = loader.load()

In [4]:
documents[-1]

Document(metadata={'producer': 'Skia/PDF m94 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'HR Guide: Policy and Procedure Template', 'source': 'HR-Guide.pdf', 'total_pages': 53, 'page': 52, 'page_label': '53'}, page_content="PRINCE EDWARD ISLAND\nEmployment Standards Act, YouthEmployment Act\nThese two Acts clarify the rights and obligationsofemployees and employers and set minimum standardstoensure individuals are treated fairly in the workplace.\nhttps://www.princeedwardisland.ca/sites/default/ﬁles/publications/web-guide_to_employment_standards_english_august_2019.pdf\nHuman Rights Act\nThe Act prohibits discrimination in areas such asemployment and services on the basis of certain personalcharacteristics or grounds (sex, race, disability...).Discrimination is the unequal, stereotypical, or prejudicialtreatment of persons.\nhttp://www.gov.pe.ca/photos/sites/humanrights/ﬁle/Workplace%20Rights-english-web.pdf\nOccupational Health and Safety Act\nThe Act and i

In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

In [6]:
chunks[1].page_content

'TABLE OF CONTENTS\nSECTION 1\nIntroduction\nObjectives of the Template 4\nObjectives of the Human Resource Policy 5\nEligibility & Scope 5\nImplementation & Monitoring 5\nFile Management and Amendments 5\nSECTION 2\nGeneral Policies\nDecent Work 6\nCode of Ethics 6\nHarassment-Free Workplace 7\nConﬁdentiality 8\nConﬂict of Interest 9\nReporting Violations 10\nSECTION 3\nEmploymentInformation\nHours of Work 11\nDress Code 12\nOvertime/Lieu Time 12\nTravel and Expense Claims 13\nStatutory Holidays 13\nPersonnel Conﬁdentiality 13\nOutside Employment 14\nParticipation in Community Activities 14\nSolicitation on Foundation Property 15\nPersonal Use of Foundation Equipment 15\nAccuracy of Records 16\nTechnology Use 16\nCell Phone Use 18\nInsurance for Employee Use of own Auto 19\nContact with media 19\nPolicies and Procedures Template - Page2'

In [7]:
len(chunks)

105

In [8]:
chunks[51]

Document(metadata={'producer': 'Skia/PDF m94 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'HR Guide: Policy and Procedure Template', 'source': 'HR-Guide.pdf', 'total_pages': 53, 'page': 27, 'page_label': '28'}, page_content='Use of Alcohol:\nThe Foundation does not provide funds for the purchaseof alcohol at Foundation sponsored activities. However,employees may consume alcohol if they so choose underthe following guidelines:\nA. Employees are expected to make travel arrangementsat functions where they are representing theFoundation and at which they are consuming alcohol, such as: assign a designated driver, takealternate transportation, arrange to stay in a hotel\nB. An employee may be asked to stop consuming alcohol if it is believed they are exhibiting inappropriatebehaviour.\nAny employee found to be in violation of the precedingparagraph will be subject to disciplinary actionwhich mayinclude termination of employment.\nPolicies and Procedures Template -

In [9]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large", chunk_size=10) # nomic-embed-text-v1 is god 

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="vectorstore/chroma_db_openai/",
    collection_name="hr_policies_v1",
)
vectordb.persist()

/tmp/ipykernel_2732/1361921395.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [10]:
del vectordb

* Reload the vector store and query the data

In [11]:
# Re-load your persisted DB
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    embedding_function=embedding_model,
    persist_directory="vectorstore/chroma_db_openai",
    collection_name="hr_policies_v1",
)

/tmp/ipykernel_2732/2285222740.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


In [12]:
query = "What are the max day offs an employee can take??"
results = vectordb.similarity_search_with_score(query, k=5)

# Inspect docs and their distances
for i, (doc, distance) in enumerate(results):
    print(f"--- Result {i} ---")
    print(f"Similarity (higher is better): {distance:.4f}")
    print("Content:", doc.page_content.replace("\n", " "), "…")
    print("Metadata:", doc.metadata)
    print()

--- Result 0 ---
Similarity (higher is better): 0.9402
Content: Sick leave will not accrue for any period during which an employee is absent from work for more than tenworkdays, except for absence due to holidays, vacation and personal days and approved Employment StandardAct leaves. When an eligible employee will be oﬀon Sick leave, he/she must notify his/her supervisoras soon ashe/she knows that he/she will be unable to work, butno later than the starting time of the employee’swork day.Employees must notify their supervisor on each dayof absence unless other arrangements have been made. When an eligible salaried employee has been out onsick leave for more than three (3) consecutive scheduledworkdays, he/she is required to submit documentation froma health care provider to their supervisor certifyingthemedical necessity for the absence and expected dateof return to work. Sick leave may be used for illnessof theeligible employee or for illness of a member of his/herhousehold (up to thr

In [13]:
retriever = vectordb.as_retriever(search_kwargs={"k": 12})

In [14]:
query = "What are the hiring policies we have?"
results = retriever.invoke(query)

# Inspect docs and their distances
for i, doc in enumerate(results):
    print(f"--- Result {i} ---")
    print("Content:", doc.page_content.replace("\n", " "))
    print("Metadata:", doc.metadata)
    print()

--- Result 0 ---
Content: SECTION 6 Hiring Hiring Process Being open and clear with candidates helps to setproper expectations, avoid disappointments, andshape a positive candidate experience. By being transparent, your Foundation will not only have aneasier time recruiting top talent, you will also havehigher rates of retention because expectations havebeen clear from the beginning. Given that the Foundationbeneﬁts the community, it is important in thehiring practices, to ensure that the workplace reﬂectstheir community and to authentically grow acommitment to gender equity as an organization. This policy provides guidelines that will help tomitigate risk in all aspects of the hiring process. Usethis as a guideline when developing your Foundation’sprocess and amend to ﬁt your practices. Policy and Procedure Statement The success of the Foundation in attempting to achieveits vision depends on having the right employees, properlytrained and motivated, applying their skills and talentsto

### HR-Policies VDB vol2

* Create the vector store

In [20]:
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_openai import OpenAIEmbeddings

from langchain.vectorstores import Chroma
import json

In [21]:
PDF_DIR = "./knowledge_base/HR-Policies V2/"

loader = DirectoryLoader(
    PDF_DIR,
    glob="**/*.txt",
    loader_cls=TextLoader,
)
documents = loader.load()

In [22]:
documents[-1]

Document(metadata={'source': 'knowledge_base/HR-Policies V2/Chapter 9 - Code of Conduct.txt'}, page_content='37signals Employee Handbook\nChapter 9:\n\nCode of Conduct\nNext: State Leave Provisions\n\nWe expect all active 37signals employees and contractors to:\n\nAssume good intentions. Approach work relationships defaulting to trust and positivity.\nWork “in the open” and be open to teaching and learning from others.\nBe respectful and empathetic, especially when it comes to differing viewpoints and experiences.\nGracefully accept constructive criticism and direct feedback, and offer feedback in the same spirit.\nWe expect 37signals to be a healthy place for all staff. 37signals prohibits all forms of discrimination and harassment.\n\nIf you experience or witness something that violates our Code of Conduct, please report it to your manager or to your People Ops team via email or chat. All reports will be reviewed and investigated, and your confidentiality will be as protected as poss

In [23]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)

In [24]:
chunks[1].page_content

'To keep everyone’s devices safe and secure, we manage all our Mac devices using Kandji as well as our in-house tool Shipshape 🔒.\n\nYour training schedules and onboarding expectations will be in your welcome project. You’ll also find docs with helpful links to technical documentation, walkthrough videos, important Basecamp projects, and more.\n\nYour welcome project will also contain a list of people to meet over the course of your first few weeks.'

In [25]:
len(chunks)

86

In [26]:
chunks[51]

Document(metadata={'source': 'knowledge_base/HR-Policies V2/Chapter 16 - Titles for Support.txt'}, page_content='Oversees performance management for CS Managers. Assists Managers with administering PIPs, performing terminations, onboarding & training new hires, recommending promotions, and managing performance lags. Able to act as ultimate authority for all team performance management and make accurate judgment calls that reflect the needs of the team & company.\nApproves and maintains oversight and controls over Support projects that add value to team skills and tooling, team/company processes, and customer experience.\nTech and Tooling\nProficient in Support stack, and uses technology appropriately to provide meaningful insights and to make recommendations. Makes informed, thoughtful decisions about new tools & services and communicates those to team/executive team as needed.\nScope\nOversees all team project work with support from Managers. Able to act as ultimate authority on proje

In [28]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large", chunk_size=10) # nomic-embed-text-v1 is god 

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="vectorstore/chroma_db_openai/",
    collection_name="hr_policies_v2",
)
vectordb.persist()

/tmp/ipykernel_2024/3573770916.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [29]:
del vectordb

* Reload the vector store and query the data

In [30]:
# Re-load your persisted DB
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vectordb = Chroma(
    embedding_function=embedding_model,
    persist_directory="vectorstore/chroma_db_openai",
    collection_name="hr_policies_v2",
)

In [31]:
query = "What are the max day offs an employee can take??"
results = vectordb.similarity_search_with_score(query, k=5)

# Inspect docs and their distances
for i, (doc, distance) in enumerate(results):
    print(f"--- Result {i} ---")
    print(f"Similarity (higher is better): {distance:.4f}")
    print("Content:", doc.page_content.replace("\n", " "), "…")
    print("Metadata:", doc.metadata)
    print()

--- Result 0 ---
Similarity (higher is better): 0.9756
Content: Paid Time Off 37signals offers 20 days of vacation and personal days plus 11 local holidays every year. We ask that you track your time off. Your vacation time is prorated based on your start date during your first year at 37signals. Your vacation time rolls over year to year with a maximum bank of 27 days at any time. If you are terminated or resign from 37signals with vacation days in your bank, you’ll be paid the monetary equivalent for those unused days (prorated at the time of your departure). Unused holidays are not paid upon termination or resignation.  Sabbatical In addition to annual PTO and holidays, every three years employees may take a 6-week paid sabbatical. Give your team a heads-up, preferably three months in advance, so they can coordinate. If you’re terminated or resign from 37signals with an unused sabbatical, 37signals will pay you the monetary equivalent of those unused days only if your unused sabbati

In [32]:
retriever = vectordb.as_retriever(search_kwargs={"k": 12})

In [33]:
query = "What are the hiring policies we have?"
results = retriever.invoke(query)

# Inspect docs and their distances
for i, doc in enumerate(results):
    print(f"--- Result {i} ---")
    print("Content:", doc.page_content.replace("\n", " "))
    print("Metadata:", doc.metadata)
    print()

--- Result 0 ---
Content: 37signals Employee Handbook Chapter 9:  Code of Conduct Next: State Leave Provisions  We expect all active 37signals employees and contractors to:  Assume good intentions. Approach work relationships defaulting to trust and positivity. Work “in the open” and be open to teaching and learning from others. Be respectful and empathetic, especially when it comes to differing viewpoints and experiences. Gracefully accept constructive criticism and direct feedback, and offer feedback in the same spirit. We expect 37signals to be a healthy place for all staff. 37signals prohibits all forms of discrimination and harassment.  If you experience or witness something that violates our Code of Conduct, please report it to your manager or to your People Ops team via email or chat. All reports will be reviewed and investigated, and your confidentiality will be as protected as possible during the investigation.  If an employee is found to have violated our Code of Conduct, it 